In [0]:
# Input files: whole dataset, paragraphs file
# ouput: word2vec model, paragraphs embeding

In [0]:
import numpy as np
from gensim.models import Word2Vec
import re
import json
import glob
import os

In [0]:
basePath = '/content/'

'''data_dir = basePath+'tempdataset'
drive_dir = basePath+'drive/My Drive/temp/tempDataset' '''

data_dir = basePath+'dataset'
drive_dir = basePath+'drive/My Drive/temp/DatasetN'

paraFile = basePath + 'paragraph1.json'

In [0]:
'''import zipfile
import os
for file_name in os.listdir(drive_dir):
  if file_name.endswith('.zip'):
    with zipfile.ZipFile(drive_dir+'/'+file_name,'r') as zip_dir:
      zip_dir.extractall(path='/content/')'''

"import zipfile\nimport os\nfor file_name in os.listdir(drive_dir):\n  if file_name.endswith('.zip'):\n    with zipfile.ZipFile(drive_dir+'/'+file_name,'r') as zip_dir:\n      zip_dir.extractall(path='/content/')"

In [0]:
def sifEmbeddings(sentences, model, alpha=1e-3):

    vocab = model.wv.vocab
    embs = model.wv       
    size = model.vector_size  
    
    total = 0
    for word in vocab:
        total += vocab[word].count 

    output = []
    
    for sent in sentences:
        count = 0
        v = np.zeros(size, dtype=np.float32) 
        for word in sent:
          if word in vocab:
            #print(word)
            weight = alpha / (alpha + (vocab[word].count/total))
            v += weight * embs[word]
            count += 1 
                
        if count > 0:
            v *= 1/count

        output.append(v)
    return np.vstack(output)

In [0]:
def splitSent(sent):
    regX = '[\s()\.\?]'
    splitSent = re.split(regX,sent)
    splitSent = list(filter(None,splitSent))
    return splitSent

In [0]:
def getDatasetAsSents():
    fileSents = []
    dirs = os.listdir(data_dir)
    print(dirs)
    for d in dirs:
      filenames = glob.glob(os.path.join(data_dir,d)+"/*json")
      for file in filenames:
          with open(file,'r') as f:
              x = json.loads(f.read())
              for option in x['options'][0]:
                  fileSents.append(option)
              fileSents.append(x['questions'][0])
              fileSents.append(x['article'])
    return fileSents

# **Word2Vec**

In [0]:
fileSents = getDatasetAsSents()
sentences = [ splitSent(sent) for sent in fileSents]

['dev', 'test', 'train']


In [0]:
model = Word2Vec(sentences, min_count=1, size=10)

In [0]:
model.save(basePath+"word2vec.model")
#model.save(drive_dir+"/word2vec.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


# **Embedding Generation**

In [0]:
f = open(paraFile,'r')
x = json.loads(f.read())
f.close()
fileSents = [ x['para'+str(id)] for id in range(1,len(x)+1)]
sentences = [ splitSent(sent) for sent in fileSents]

In [0]:
model = Word2Vec.load("word2vec.model")
embs = sifEmbeddings(sentences, model, alpha=1e-3)
print(embs.shape)
np.savetxt(basePath+'paraEmbs.csv',embs, delimiter=',',fmt='%.15f')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


(10527, 10)
